## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [6]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
             'OverTime', 'StockOptionLevel', 'WorkLifeBalance',
             'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y_df, test_size=0.2, random_state=42)


In [9]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Checking Unique Values
print(X_train['OverTime'].value_counts())

OverTime
No     837
Yes    339
Name: count, dtype: int64


In [12]:
from sklearn. preprocessing import OneHotEncoder
# OneHotEncoder for OverTime
overtime_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit -> Training Data
overtime_encoder.fit(X_train[['OverTime']])

OneHotEncoder(drop='first', sparse_output=False)

In [15]:
# Transforming T-Data
overtime_encoded_train = overtime_encoder.transform(X_train[['OverTime']])
overtime_encoded_test = overtime_encoder.transform(X_test[['OverTime']])

# Column Names
overtime_columns = ['OverTime_Yes']

# Dropping Original OverTime Column and Adding Encoded Columns
X_train_encoded = X_train.drop('OverTime', axis=1)
X_test_encoded = X_test.drop('OverTime', axis=1)

# Encoded Column's
X_train_encoded = X_train_encoded.join(
    pd.DataFrame(overtime_encoded_train, columns=overtime_columns, index =X_train.index))
X_test_encoded = X_test_encoded.join(
    pd.DataFrame(overtime_encoded_test, columns=overtime_columns, index=X_test.index))

# Verifying the data is now all numeric
print("\nData types afrer encoding:")
print(X_train_encoded.dtypes)


Data types afrer encoding:
Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTime_Yes               float64
dtype: object


In [16]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [32]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = dept_encoder.transform(y_train[['Department']])
department_test_encoded = dept_encoder.transform(y_test[['Department']])

print("Department encoding shapes:")
print("department_train_encoded shape:", department_train_encoded.shape)
print("y_train shape:", y_train.shape)

Department encoding shapes:
department_train_encoded shape: (1176, 3)
y_train shape: (1176, 2)


In [34]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attr_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attr_encoder.transform(y_test[['Attrition']])


## Part 2: Create, Compile, and Train the Model

In [35]:
# Find the number of columns in the X training data.
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu')(input_layer)
shared1 = layers.Dropout(0.3)(shared1)
shared2 = layers.Dense(128, activation='relu')(shared1)
shared2 = layers.Dropout(0.3)(shared2)

In [36]:
# Create a branch for Department
# with a hidden layer and an output layer
department_hidden = layers.Dense(32, activation='relu')(shared2)

# Create the hidden layer
department_hidden = layers.Dropout(0.2)(department_hidden)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)

In [37]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_hidden = layers.Dense(32, activation='relu')(shared2)
# Create the hidden layer
attrition_hidden = layers.Dropout(0.2)(attrition_hidden)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)

In [43]:
# Create the model
model = Model(inputs=input_layer, outputs=[attrition_output, department_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'accuracy']
)

# Summarize the model
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 64)             │            704 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 64)             │              0 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 128)            │          8,320 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 128)            │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 32)             │          4,128 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 32)             │          4,128 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 32)             │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 32)             │              0 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ dropout_6[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
# Train the model
history = model.fit(
    X_train_scaled,
    [attrition_train_encoded, department_train_encoded],
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - attrition_output_accuracy: 0.5802 - attrition_output_loss: 0.6499 - department_output_accuracy: 0.5366 - department_output_loss: 1.0089 - loss: 1.6587 - val_attrition_output_accuracy: 0.7966 - val_attrition_output_loss: 0.5135 - val_department_output_accuracy: 0.6356 - val_department_output_loss: 0.8224 - val_loss: 1.3425
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - attrition_output_accuracy: 0.8409 - attrition_output_loss: 0.4616 - department_output_accuracy: 0.6326 - department_output_loss: 0.8173 - loss: 1.2785 - val_attrition_output_accuracy: 0.7966 - val_attrition_output_loss: 0.4471 - val_department_output_accuracy: 0.6356 - val_department_output_loss: 0.8107 - val_loss: 1.2694
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - attrition_output_accuracy: 0.8462 - attrition_output_loss: 0.4145 - department_output_accuracy: 0.6508 - department_output_loss: 0.7833 - loss: 1.1983 - val_attrition_output_accuracy: 0.7966 -

In [45]:
# Evaluate the model with the testing data
test_results = model.evaluate(
    X_test_scaled,
    [attrition_test_encoded, department_test_encoded],
    verbose=1
)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8505 - attrition_output_loss: 0.4714 - department_output_accuracy: 0.6032 - department_output_loss: 0.8919 - loss: 1.3696 


In [46]:
# Print the accuracy for both department and attrition
print(f"Attrition predictions accuracy: {test_results[1]}")
print(f"Department predictions accuracy: {test_results[2]}")

Attrition predictions accuracy: 0.4186553955078125
Department predictions accuracy: 0.8615788221359253


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy isn't the best metric here because the attrition data is probably unbalanced. Most employees typically stay at a company rather than leave.
If only 15% of employees leave in our dataset, a model that always predicts "No" would be 85% accurate without learning anything useful! That's misleading.
Better options would be metrics like precision/recall that focus on correctly identifying the employees who are likely to leave, which is usually what companies care most about.

2. I used softmax for both output layers because:
For department prediction: Softmax works well when predicting categories (like which of 3 departments someone belongs to). It gives us percentages that add up to 100%, so we can see which department is most likely.
For attrition prediction, Even though it's just Yes/No, softmax still works great by giving us the probability of someone leaving versus staying.

3.
Handling the imbalanced data better - maybe giving more weight to the "Yes" attrition examples during training
Adding more relevant features that might predict why people leave, like combining salary with distance from home
Trying different layer sizes and dropout rates to find the sweet spot
